## Álgebra Lineal Computacional - 2C 2022


### Cuadrados mínimos - Regresión lineal

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Ejemplo 

Retomemos el ejercicio del martes pasado para incorporar los comandos.
Buscamos los parámetros $v_0$ y $k$ que mejor ajustaran (en el sentido de mínimos cuadrados) la ecuación de la señal eléctrica $V(t)=v_0 e^{-\frac{t}{k}}$ a la tabla:


| t | 1  | 2   | 3   | 4   | 5   |
|---|----|-----|-----|-----|-----|
| V | 0,4| 0,25| 0,17| 0,07| 0,05|

El problema viene dado por una función exponencial, por lo que linealizamos el problema tomando logaritmo. 

$$y=ln(V(t))=-\frac{1}{k} t + ln(v_0) \Longrightarrow y=a_1t+a_0$$
y buscamos $a_1$ y $a_0$ que mejor ajustaran la nueva tabla:

| t |  1    |  2     |   3    |   4    |  5     |
|---|-------|--------|--------|--------|--------|
| y |ln(0,4)|ln(0,25)|ln(0,17)|ln(0,07)|ln(0,05)|

Lo resolvimos buscando la solución por cuadrados mínimos de  
$$Ax=b$$  
$$A=\left(\begin{matrix} 1 & 1 \\ 2 & 1 \\ 3 & 1 \\ 4 & 1 \\ 5 & 1 \end{matrix} \right), \quad x=\left(\begin{matrix} a_1 \\ a_0 \end{matrix} \right), \quad b=\left(\begin{matrix} ln(0,4) \\ ln(0,25) \\ ln(0,17) \\ ln(0,07) \\ ln(0,05) \end{matrix} \right)$$.
Reveamos las tres formas de resolución planteadas:


In [ ]:
# Datos del problema
t = np.array([1, 2, 3, 4, 5])
v = np.array([0.4,0.25, 0.17, 0.07, 0.05])
y=np.log(v)

# Construyo A por columnas.
unos=np.ones(len(t)) #la columna de unos

#usamos np.c_[] que ya lo habíamos usado hace tiempo
A = np.c_[t]
A= np.c_[A,unos]  #también puede ser útil np.vstack para ir pegando vectores

b = y #con los logaritmos!!!

### Forma 1: 
Resolver las ecuaciones normales:
$A^TA.x=A^Tb$ 

In [ ]:
x = np.linalg.solve(A.T@A, A.T@b)  # Obtenemos la solución del sistema
print(x)
y = np.poly1d(x) #reconstruye el polinomio. ATENCIÓN CON EL ORDEN DE LOS COEFICIENTES
print(y)

Grafiquemos los datos junto con la solución obtenida:

In [ ]:
xs = np.linspace(t[0]-0.25, t[-1]+0.25, 100) #nos creamos este array auxiliar para hacer los gráficos 

plt.plot(xs, y(xs), c='r') #evalúo el polinomio en el array auxiliar
plt.scatter(t, b)          #los puntos

Recordamos que 
$$a_1=-\frac{1}{k} \Longrightarrow k=-\frac{1}{a_1} \quad \mbox{y}\quad a_0=ln(v_0) \Longrightarrow v_0=e^{a_0}$$
y llegamos al resultado de la clase 
$$V(t)=v_0 e^{-\frac{t}{k}}$$
Grafiquemos ahora la aproximación a la tabla original obtenida:

In [ ]:
k=-1/x[0]
v_o=np.exp(x[1])
V= lambda t: v_o*np.exp(-t/k) #nuestra función
plt.plot(xs,V(xs),c='r')      
plt.scatter(t,v)              #los datos de la tabla sin linealizar

### Forma 2: 
#### DVS reducida

Para resolver $Ax=b$ recordamos que si la DVS de $A$ es $A = \hat{U}\hat{\Sigma}\hat{V}^t$ sabemos que la solución por cuadrados mínimos que buscamos es 
$$x = A^\dagger b$$
Donde $A^\dagger=\hat{V}\hat{\Sigma}^{-1}\hat{U}^t$ es la pseudo inversa de $A$.

In [ ]:
x=np.linalg.pinv(A)@b  #np.linalg.pinv(A) nos la A DAGUITA!!!!
print(x)               # se obtiene la misma solución! 

Y el ejercicio se concluye igual que antes recuperando la función $V(t)$ original.

### Forma 3: 

#### Usando ``np.polyfit``

Obtenemos $y$ usando ``np.polyfit`` que ya vieron en la teórica que les da el polinomio que mejor ajusta por cuadrados mínimos. 

Observar que en este caso no hace falta construir la matriz $A$, directamente cargamos los datos linealizados y usamos la función de Numpy:

In [ ]:
# polyfit(x, y, n) calcula el polinomio de grado n que mejor ajusta los datos (x_i, y_i) por mínimos cuadrados
# Nos devuelve el vector de coeficientes empezando por la potencia de grado más alto
x = np.polyfit(t, b, 1)
print(x)               #chequeadísimo

## Ejercicio

En el archivo ``emisiones_CO2.csv`` se encuentran los datos de las emisiones de $CO^2$ (en toneladas per cápita) de Argentina de 1960 a 2018. 

Se desea hallar la función $f$ de la forma
$$f(x)=\alpha_1\cos\left(\dfrac{x}{18}\right) + \alpha_2{x}$$
que mejor ajuste esos datos en el sentido de cuadrados mínimos.

En este caso, tenemos que $f$ no es polinomial ni linealizable. 
Recordar que en general si llamamos 

$$f_1(x) = \cos\left(\dfrac{x}{18}\right) ,\qquad f_2(x) = x$$

buscamos $\alpha_1$ y $\alpha_2$ que minimicen $\displaystyle \sum_{i=1}^{59} |\alpha_1 f_1(x_i) + \alpha_2 f_2(x_i) - y_i|^2$

**Como $f$ NO es un polinomio, NO podemos usar ``np.polyfit``**. Tenemos que usar alguna de las otras formas. 

Empezamos leyendo los datos de ``emisiones_CO2.csv``:

In [ ]:
# Cargo los datos desde el archivo
data_frame = pd.read_csv('emisiones_CO2.csv')
data_frame.head(8)  #vemos los primeros 8 datos

In [ ]:
# Lo convierto a una matriz de numpy
xy = data_frame.to_numpy(dtype='float')
print(xy[:8])

In [ ]:
# Graficamos los datos de la tabla. x es la primera columna de xy, y es la segunda
x = xy[:, 0]
y = xy[:, 1]

# Visualizo los datos
plt.scatter(x,y)

#### a)
Construir la matriz A y b del problema y resolver el sistema $Ax=b$ con la técnica de cuadrados mínimos:

In [ ]:
A= ###completar. Pensar qué tiene en sus columnas, para la primera columna necesitarán hacer la función cos(x/18)...
b= ###completar
alpha= ###solución por cad mín de Ax=b

#### b)
Graficar la función $f$ obtenida junto con los datos.

In [ ]:
xs=###completar (recordar linspace!)
###completar!

#### c)
Estimar las emisiones emitidas en 2019 (el dato real es de 4,06 toneladas).
(Recordar que la tabla tiene 58 filas) 

In [ ]:
###completar

## Ejercicio (ej. 11 P7)

Implementar un programa que reciba como input una lista de funciones $\{f_1, \dots , f_n\}$ y dos vectores $x= (x_1, \dots, x_n)$, $y=(y_1, \dots, y_n)$ y calcule la función:
$$f=\displaystyle\sum_{i=1}^{n} \alpha_if_i(x)$$
que mejor aproxima los datos en el sentido de cuadrados mínimos.

In [ ]:
def cuad_min(funciones,x,y):
    At = []                # Acá voy a armar la matriz A por filas, luego debo transponerla
    for f in funciones:
        filai=f(x)
        At.append(filai)
    
    A =  (np.array(At)).T     # At es una lista de listas. La convierto en array y luego transpongo
    
    alpha = np.linalg.pinv(A)@y # Resuelvo Ax = b
    
    f = lambda x: sum([alpha[i]*funciones[i](x) for i in range(len(funciones))]) # Armo f como combinación lineal de las fi
    
    return(alpha,f) # Además de la función devuelvo los coeficientes, para poder tenerlos a mano

probemos nuestro programa con el ejercicio anterior:

In [ ]:
f1= lambda t: np.cos(t/18)
f2= lambda t: t
f=[f1,f2]

print(cuad_min(f,x,y)[0])

print(cuad_min(f,x,y)[1](0)) #evalúo la función construida en t=0 para verificar